In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import ImageFile, Image
import warnings
import boto3

In [3]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from PIL import ImageFile, Image
import io
# Initialize S3 client
session = boto3.Session(
#    aws_access_key_id='',
#    aws_secret_access_key='' 
)
# client = session.client('s3')
s3 = session.client('s3')
s3_bucket = '207-images-labels'
def read_data_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    data = np.load(io.BytesIO(response['Body'].read()))
    return data
def write_data_to_s3(bucket, key, data):
    np_bytes = io.BytesIO()
    np.save(np_bytes, data)
    np_bytes.seek(0)
    s3.put_object(Bucket=bucket, Key=key, Body=np_bytes)
def list_objects_in_s3(bucket, prefix):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    if 'Contents' in response:
        return [obj['Key'] for obj in response['Contents']]
    else:
        return []

In [14]:
images = list_objects_in_s3(s3_bucket, 'train/pre1/images/img_numpy_train_batch_1')

In [19]:
eval_images = []
for i in range(8):
    print(i+1)
    images = list_objects_in_s3(s3_bucket, f'train/pre1/images/img_numpy_train_batch_{i+1}')
    eval_images += ([x for x in images if x not in lines])
print(len(eval_images))
with open('evaluate_images', 'w') as f:
    for item in eval_images:
        f.write("%s\n" % item)

1
2
3
4
5
6
7
8
2582


In [8]:
#Formatting test/validation data to evaluate model
# Define the file path
file_path = '../train_images'

# Open the file and read lines into a list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Remove trailing newline characters and extra whitespace from each line
lines = [line.strip() for line in lines]

# Print the list to verify
images_not_trained = [x for x in images if x not in lines]
print(len(images_not_trained))
with open('evaluate_images', 'w') as f:
    for item in images_not_trained:
        f.write("%s\n" % item)

686


In [15]:
print(len(images))
images_not_found = [x for x in lines if x not in images]
print(images_not_found)

1000
['train/pre1/images/img_numpy_train_batch_190_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1913_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1920_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1921_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1926_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1931_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1933_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1939_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1941_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1947_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1954_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1958_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1964_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1977_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1981_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1991_imgs.npy', 'train/pre1/images/img_numpy_train_batch_1993_imgs.npy', 'train/pre1/images/img_num

In [4]:
# Load the model
model = tf.keras.models.load_model('baseline.h5' , compile=False)
class CustomAdam(tf.keras.optimizers.Adam):
    def __init__(self, *args, **kwargs):
        super(CustomAdam, self).__init__(*args, **kwargs)

    def get_config(self):
        config = super(CustomAdam, self).get_config()
        return config

# Register the custom optimizer in TensorFlow's custom objects
tf.keras.utils.get_custom_objects().update({'Custom>Adam': CustomAdam})
# Display the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 998, 998, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 499, 499, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 7968032)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               1019908224
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,019,909,249
Trainable params: 1,019,909,249
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

def evaluate_batch(batch_name):
    imgs = read_data_from_s3(s3_bucket, f'train/pre1/images/img_numpy_train_batch_{batch_name}_imgs.npy')
    lbls = read_data_from_s3(s3_bucket, f'train/pre1/labels/img_numpy_train_batch_{batch_name}_lbls.npy')
    loss, accuracy = model.evaluate(imgs, lbls)
    return accuracy

In [21]:
#evaluate batch and dump acc scores to file
import json
try:
    with open('image_accuracy','r') as writing:
        acc = json.load(writing)
except:
    acc = {}
count = 0
tot = 300
with open('evaluate_images', 'r') as read_file:
        for i in read_file:
            if count > tot:
                print(i[40:-10])
                acc[i] = evaluate_batch(i[40:-10])
            count += 1
            if (count%(tot+50)) == tot+50-1:
                break
                
with open('image_accuracy','w') as writing:
    json.dump(acc, writing)
        

137
1/1 [==============================] - 2s 2s/step - loss: 0.1975 - accuracy: 0.6500
1380
1/1 [==============================] - 2s 2s/step - loss: 0.1994 - accuracy: 0.7000
1381
1/1 [==============================] - 2s 2s/step - loss: 0.2147 - accuracy: 0.6000
1382
1/1 [==============================] - 2s 2s/step - loss: 0.1455 - accuracy: 0.9000
1384
1/1 [==============================] - 2s 2s/step - loss: 0.1318 - accuracy: 0.8000
1387
1/1 [==============================] - 2s 2s/step - loss: 0.1796 - accuracy: 0.8000
1389
1/1 [==============================] - 2s 2s/step - loss: 0.1705 - accuracy: 0.7500
138
1/1 [==============================] - 2s 2s/step - loss: 0.1621 - accuracy: 0.7500
1390
1/1 [==============================] - 2s 2s/step - loss: 0.2454 - accuracy: 0.6000
1391
1/1 [==============================] - 2s 2s/step - loss: 0.1544 - accuracy: 0.6500
1392
1/1 [==============================] - 2s 2s/step - loss: 0.1381 - accuracy: 0.9000
1393
1/1 [=============

In [20]:
#sum all accuracy scores
import json
acc = 0
with open('image_accuracy','r') as json_file:
    data = json.load(json_file)
count = 0
for key in data:
    acc += float(data[key])
    count += 1
print(acc/count, count)

0.7551229516746568 244


In [11]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import boto3


# Function to evaluate a batch and collect predictions and labels
def evaluate_batch(batch_name):
    imgs = read_data_from_s3(s3_bucket, f'train/pre1/images/img_numpy_train_batch_{batch_name}_imgs.npy')
    lbls = read_data_from_s3(s3_bucket, f'train/pre1/labels/img_numpy_train_batch_{batch_name}_lbls.npy')
    
    # Assuming lbls are binary (0 or 1)
    lbls = lbls.astype(int)
    
    # Get predictions
    probabilities = model.predict(imgs)
    predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary class labels
    
    return predictions, lbls

# Collect all predictions and labels
all_predictions = []
all_labels = []

for i in range(1, 11):  # Assuming batch names are from 1 to 10
    batch_predictions, batch_labels = evaluate_batch(i)
    all_predictions.extend(batch_predictions)
    all_labels.extend(batch_labels)

# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[106  12]
 [ 19  63]]


In [14]:
all_predictions = []
all_labels = []
count = 0
with open('evaluate_images', 'r') as read_file:
        for i in read_file:
            batch_predictions, batch_labels = evaluate_batch(i[40:-10])
            all_predictions.extend(batch_predictions)
            all_labels.extend(batch_labels)
            if (count == 50-1):
                break
            count += 1
conf_matrix = confusion_matrix(all_labels, all_predictions)

print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[505 101]
 [154 240]]


In [17]:
print((505+240)/(505+240+154+101))

0.745
